In [1]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

1- 10.28  
2- 58  
3- 

In [11]:
year = 2022
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [17]:
def save_results(df, y_pred, output_file):
    df_result = pd.DataFrame()
    df_result['ride_id'] = df.ride_id
    # df_result['tpep_pickup_datetime'] = df.tpep_pickup_datetime
    # df_result['tpep_dropoff_datetime'] = df.tpep_dropoff_datetime
    # df_result["PULocationID"] = df.PULocationID
    # df_result["DOLocationID"] = df.DOLocationID
    # df_result["duration"] = df.duration
    df_result["duration_predicted"] = y_pred
    # df['prediction_error'] = df_result.duration_predicted - df_result.duration
    df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
    )

In [18]:
print('predicted mean duration:', y_pred.mean())
save_results(df, y_pred, 'predictions.parquet')